In [2]:
import pandas as pd
import pickle
import requests
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder

data_url = 'https://raw.githubusercontent.com/tomweinandy/AIinPowerBI/main/vending_revenue.csv'
algo_url = 'https://github.com/tomweinandy/AIinPowerBI/blob/main/vending_model.pickle?raw=true'

# Load data
# df = pd.read_csv('vending_revenue.csv')
df = pd.read_csv(data_url)
df = df.dropna()
df = df.drop(columns=['WeekStartingSat'])

# # Load the saved model from the pickle file
# with open('vending_model.pickle', 'rb') as file:
#     model_info = pickle.load(file)

# Load the pickled deep learning algorithm
response = requests.get(algo_url)
model_info = pickle.loads(response.content)

# Reconstruct the model architecture
loaded_model = tf.keras.models.model_from_json(model_info['architecture'])

# Set the model weights
loaded_model.set_weights(model_info['weights'])

# Preprocess the data
label_encoder = LabelEncoder()
df['Location'] = label_encoder.fit_transform(df['Location'])  # Encode location labels

# Split the data into input features (X)
X = df.drop('Location', axis=1)

# Predict the labels
predictions = loaded_model.predict(X)

# Change predictions to integers between 0 and 4
encoded_labels = [int(min(max(p.round(0), 0), 4)) for p in predictions]

# Decode the predicted labels
decoded_labels = label_encoder.inverse_transform(encoded_labels)

# Add the predicted labels to the DataFrame
df['Predicted_Location'] = decoded_labels

# Decode Location
df['Location'] = label_encoder.inverse_transform(df['Location'])

# Print accuracy
accuracy = sum(df['Location'] == df['Predicted_Location'])/len(df)
print(f"\nAccuracy: {accuracy}")

# Print the DataFrame with the predictions
df[['Location', 'Predicted_Location']]

8/8 [==============================] - 0s 2ms/step

Accuracy: 0.9365079365079365


,Location,Predicted_Location
0,Factory,Mall 1
2,Mall 1,Mall 1
3,Mall 2,Mall 2
4,Office,Office
5,Factory,Factory
...,...,...
257,Mall 1,Mall 1
258,Mall 2,Office
259,Office,Office
262,Mall 1,Office
